# Excercise 6: Monitoring of our model

## 1. Serve Model using Seldon Core


### Create Dummy Seldon Core deployment

Create a new file in the folder `pipelines` named `seldon-model.yml`:

```yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: <yourname>-model
spec:
  name: <yourname>-model-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: classifier
          image: fluescher/seldon-dummy-service
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: eligibility
    labels:
      version: v1
    replicas: 1
```

Replace `<yourname>` with your name.

This will create a new seldon deployment that is exposed over the Api Gateway. Check the deployment state of your model:

`kubectl get pods`

If you see your deployment running you can continue testing.

### Test deployment

Our dummy deployment can now be accessed using the api gateway. Execute the following line after replacing `<yourname>` with your name.

You should see something like `{"data":{"names":[],"ndarray":[1]},"meta":{"requestPath":{"classifier":"...."}}}`
which is the result of your modelwhich is the result of your model.



In [ ]:
!wget -O- mlproduction.dsiag.ch/seldon/<yourname>/<yourname>-model/api/v1.0/predictions --post-data '{"data": { "ndarray": [[25000, 189625]]}}' --header='Content-Type:application/json'

### Check default metrics

Right after deployment, a selection of basic metrics is already collected by Seldon. 

Open Grafana: http://monitoring.mlproduction.dsiag.ch

User: admin
Password: password

To create load, you can execute the following line, after replacing `<yourname>`.


In [ ]:
!NAMESPACE=<yourname> MODEL=<yourname>-model ../services/create-load.sh

Questions: 
- Which metrics are collected? 
- Are there metrics that are visible in the dashboard but do not update?


### Query Metrics

You can query metrics by clicking on _Explore_ in the Grafana menu. 

To query metrics you can enter a Prometheus query in the search bar.

Have a look at what you can do: https://prometheus.io/docs/prometheus/latest/querying/basics/

### Bonus: Reconfigure Kubernetes deployment

If you want that your Sustaino frontend uses the newly deployed service, you need to change `pipeline/frontend.yml`

Change

```yaml
env:
    - name: ELIGIBILITY_API_ENDPOINT
      value: http://eligibility-service:8100/predict
```

to:

```yaml
env:
    - name: ELIGIBILITY_API_ENDPOINT
      value: http://mlproduction.dsiag.ch/seldon/<yourname>/<yourname>-model/api/v1.0/predictions
```


Remember to replace `<yourname>`

Apply the configuration using `kubectl apply -f pipeline`

## Custom Metrics

You find the deployed model definition in the folder `services/seldon-dummy`.

You can see that it defines two custom metrics. Are you able to find them in Grafana?

You can query multiple metrics using the Grafana Query: `{ __name__=~"metric1|metric2" }`

### Update your real eligibility model

Update your eligibility model to behave like the seldon-dummy service:

1. Return custom metrics
2. Rebuild it using Cloud Build
3. Update your  `seldon-model.yml` to use your image instead of the dummy image
4. Reapply your `seldon-model.yml`. Verify that the model does not alway return **1**

